<a href="https://colab.research.google.com/github/tkasod2/TIL/blob/main/sql/sqlite3%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3

In [ ]:
# print(sqlite3.sqlite_version)
# print(sqlite3.version)

3.22.0
2.6.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dbpath = "/content/drive/MyDrive/sql/sample.db"
conn = sqlite3.connect(dbpath)

In [ ]:
cur = conn.cursor()

# cur.execute(하나의 SQL문~~)
# cur.executescript(하나 이상의 SQL문)
# cur.executemany(~~~, ~~~) --> Data만 바꿔가면서 동일한 SQL문을 반복시행

# * SELECT문 실행 후 적용 진행
# cur.fetchone() --> 하나의 행을 불러옴(출력)
# cut.fetchall() --> 모든 행을 불러옴(출력)

## 기본 셋팅

In [ ]:
script = """
-- 1) employees 테이블이 이미 있다면 제거
DROP TABLE IF EXISTS employees;

-- 2) employees 테이블 생성 
-- 필드명, (필드)데이터 타입, 제약조건(constraints ;ex. AUTOINCREMENT, NOT NULL)
CREATE TABLE employees( 
id INTEGER PRIMARY KEY AUTOINCREMENT, --숫자를 저장, 이 테이블의 주요 Key(기본 키)로 지정, 값을 따로 입력하지 않으면 자동 증가 숫자 부여
name TEXT NOT NULL, --빈 값이 저장되는 것을 허용하지 않음
salary REAL, --소수점이 들어간 자료형
department TEXT,
position TEXT,
hireDate TEXT);

-- 3) employees 테이블에 데이터(row == record == instance) 넣기
-- Format) INSERT INTO 테이블명(필드명, 필드명, ...) VALUES(실제값, 실제값, ...) <- 지정한 필드의 수와 넣어주는 값의 갯수가 동일해야 함

INSERT INTO employees(name, salary, department, position, hireDate) VALUES('Dave', 300, 'Marketing', 'LV1', '2020-01-01');
INSERT INTO employees(name, salary, department, position, hireDate) VALUES('Clara', 420, 'Sales', 'LV2', '2018-01-11');
INSERT INTO employees(id, name, salary, department, position, hireDate) VALUES(3, 'Jane', 620, 'Developer', 'LV4', '2015-11-01');

-- 전체 필드에 빠짐없이 값을 넣을 시 필드명 생략 가능 
INSERT INTO employees VALUES(4, 'Peter', 530, 'Developer', 'LV2', '2020-11-01'); 
"""

cur.executescript(script) 
conn.commit()

In [ ]:
data = [('Elena', 510, 'Recruiter', 'LV3', '2020-07-01'), 
        ('Sujan', 710, 'HR', 'LV5', '2014-06-01'),
        ('Jake', 210, 'CEO', 'LV8', '2012-01-01')]

cur.executemany("INSERT INTO employees(name, salary, department, position, hireDate)\
                VALUES(?, ?, ?, ?, ?);", data)
conn.commit()

In [ ]:
cur.execute("SELECT * FROM employees;")
employee_list = cur.fetchall()
for employee in employee_list:
    print(employee)

(1, 'Dave', 300.0, 'Marketing', 'LV1', '2020-01-01')
(2, 'Clara', 420.0, 'Sales', 'LV2', '2018-01-11')
(3, 'Jane', 620.0, 'Developer', 'LV4', '2015-11-01')
(4, 'Peter', 530.0, 'Developer', 'LV2', '2020-11-01')
(5, 'Elena', 510.0, 'Recruiter', 'LV3', '2020-07-01')
(6, 'Sujan', 710.0, 'HR', 'LV5', '2014-06-01')
(7, 'Jake', 210.0, 'CEO', 'LV8', '2012-01-01')


In [ ]:
print([ col[0] for col in cur.description ])

['id', 'name', 'salary', 'department', 'position', 'hireDate']


In [ ]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM employees;", conn) 
df

DatabaseError: ignored

In [ ]:
conn.close()

In [ ]:
a = (1051,1051,1051,1051,1051,2073,2073,2073,2073,3063,3063,3063,3053,3024,3024)
b = ('aBxD','aBxD','aBxD','aBxD','aBxD','bDbD','bDbD','bDbC','bDbC','aaaa','aaab','aaac', '5DaD','aDDD','aDDE')
c = ('2021-10-31','2021-10-31','2021-10-31','2021-11-01','2021-11-01','2021-10-31','2021-10-31','2021-10-30', '2021-10-29','2021-11-07','2021-11-08','2021-11-09','2021-11-09','2021-11-09','2021-11-10')
df = pd.DataFrame({"user_id":a, "session_id":b, "date": c})
df

,user_id,session_id,date
0,1051,aBxD,2021-10-31
1,1051,aBxD,2021-10-31
2,1051,aBxD,2021-10-31
3,1051,aBxD,2021-11-01
4,1051,aBxD,2021-11-01
5,2073,bDbD,2021-10-31
6,2073,bDbD,2021-10-31
7,2073,bDbC,2021-10-30
8,2073,bDbC,2021-10-29
9,3063,aaaa,2021-11-07


In [ ]:
dbpath = "/content/drive/MyDrive/sql/user.db"
conn = sqlite3.connect(dbpath)

In [ ]:
# Save
df.to_sql('user',conn)

In [ ]:
# Load
df = pd.read_sql('SELECT * FROM user;', conn, index_col = 'index')

In [ ]:
cur = conn.cursor()

cur.execute('''
SELECT * FROM user
''')
# cur.executescript(하나 이상의 SQL문)
# cur.executemany(~~~, ~~~) --> Data만 바꿔가면서 동일한 SQL문을 반복시행

# * SELECT문 실행 후 적용 진행
# cur.fetchone() --> 하나의 행을 불러옴(출력)
# cut.fetchall() --> 모든 행을 불러옴(출력)

cur.fetchall()

[(0, 1051, 'aBxD', '2021-10-31'),
 (1, 1051, 'aBxD', '2021-10-31'),
 (2, 1051, 'aBxD', '2021-10-31'),
 (3, 1051, 'aBxD', '2021-11-01'),
 (4, 1051, 'aBxD', '2021-11-01'),
 (5, 2073, 'bDbD', '2021-10-31'),
 (6, 2073, 'bDbD', '2021-10-31'),
 (7, 2073, 'bDbC', '2021-10-30'),
 (8, 2073, 'bDbC', '2021-10-29'),
 (9, 3063, 'aaaa', '2021-11-07'),
 (10, 3063, 'aaab', '2021-11-08'),
 (11, 3063, 'aaac', '2021-11-09'),
 (12, 3053, '5DaD', '2021-11-09'),
 (13, 3024, 'aDDD', '2021-11-09'),
 (14, 3024, 'aDDE', '2021-11-10')]

## Data 만들기

In [ ]:
a = (1051,1051,1051,1051,1051,2073,2073,2073,2073,3063,3063,3063,3053,3024,3024)
b = ('','aBxD','aBxD','aBxD','aBxD','bDbD','bDbD','bDbC','bDbC','aaaa','aaab','aaac', '5DaD','aDDD','aDDE')
c = ('2021-10-31','2021-10-31','2021-10-31','2021-11-01','2021-11-01','2021-10-31','2021-10-31','2021-10-30', '2021-10-29','2021-11-07','2021-11-08','2021-11-09','2021-11-09','2021-11-09','2021-11-10')
df = pd.DataFrame({"user_id":a, "session_id":b, "date": c})

In [ ]:
dbpath = "/content/drive/MyDrive/sql/user_log.db"
conn = sqlite3.connect(dbpath)

In [ ]:
# Save
df.to_sql('user',conn)

In [ ]:
DRIVE_PATH = "/content/drive/MyDrive/" # 내 드라이브의 경로이다

FILE_PATH_IN_MY_DRIVE = "공부/zero_base_data_analysis/[바로-써먹는-데이터-분석]-강의자료-new4/2부/Part 1. 우리 서비스에 어떤 유저가 있어요_ - 유저 데이터 분석/Project_1_실전 프로젝트 1. 유저 행동 데이터로 서비스 건강성 분석하기/data/event_log.csv" # 내 드라이브 내 파일이 있는 경로
PATH = DRIVE_PATH +  FILE_PATH_IN_MY_DRIVE

df = pd.read_csv(PATH) # csv 파일 읽어오기
len(df)

885129

In [ ]:
pd.to_datetime(df2['event_time'])

0      2020-12-29 11:09:37+00:00
1      2021-01-26 18:59:38+00:00
2      2020-11-25 06:35:54+00:00
3      2021-02-08 12:12:04+00:00
4      2020-09-28 10:23:12+00:00
                  ...           
8846   2020-11-06 09:24:48+00:00
8847   2021-01-13 12:37:50+00:00
8848   2020-11-21 20:08:30+00:00
8849   2021-02-16 07:13:38+00:00
8850   2021-01-12 11:11:58+00:00
Name: event_time, Length: 8851, dtype: datetime64[ns, UTC]

In [ ]:
df2 = df.sample(frac = 0.01).reset_index(drop = True)
df2['event_time'] = pd.to_datetime(df2['event_time'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8851 entries, 0 to 8850
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   event_time     8851 non-null   datetime64[ns, UTC]
 1   event_type     8851 non-null   object             
 2   product_id     8851 non-null   int64              
 3   category_id    8851 non-null   int64              
 4   category_code  6429 non-null   object             
 5   brand          6784 non-null   object             
 6   price          8851 non-null   float64            
 7   user_id        8851 non-null   int64              
 8   user_session   8851 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(3), object(4)
memory usage: 622.5+ KB


In [ ]:
df2.to_sql('user_data',conn)

In [ ]:
df = pd.read_sql_query(query, conn)
df = df.iloc[:,1:]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8851 entries, 0 to 8850
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   event_time     8851 non-null   object 
 1   event_type     8851 non-null   object 
 2   product_id     8851 non-null   int64  
 3   category_id    8851 non-null   int64  
 4   category_code  6533 non-null   object 
 5   brand          6744 non-null   object 
 6   price          8851 non-null   float64
 7   user_id        8851 non-null   int64  
 8   user_session   8850 non-null   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 622.5+ KB


In [ ]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 12:18:27 UTC,view,661128,2144415925732901052,appliances.kitchen.juicer,None,63.92,1515915625519341967,TTK47Aa942
1,2020-09-24 12:50:00 UTC,cart,4013079,2144415921932861531,electronics.telephone,None,11.27,1515915625519258287,7DGcRi1GB8
2,2020-09-24 13:31:18 UTC,view,453844,2144415921857364057,None,multitronics,20.38,1515915625519421807,rTLaBXtgjK
3,2020-09-24 13:31:27 UTC,view,4027952,2144415922662670447,computers.ebooks,pocketbook,257.87,1515915625447699065,lhuHFEJkSq
4,2020-09-24 13:33:54 UTC,view,1413245,2144415929943982381,electronics.audio.microphone,maono,20.30,1515915625519423883,0nGjhb2Hfj


In [ ]:
df[df['brand']=='multitronics']

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2,2020-09-24 13:31:18 UTC,view,453844,2144415921857364057,None,multitronics,20.38,1515915625519421807,rTLaBXtgjK
1246,2020-10-21 12:34:01 UTC,view,453833,2144415941578981993,computers.desktop,multitronics,62.84,1515915625517090468,nlJCXeCicv
3286,2020-11-22 04:28:01 UTC,view,453827,2144415941578981993,computers.desktop,multitronics,126.87,1515915625542260146,alOrFsJ7IT
5404,2021-01-03 21:52:28 UTC,view,453827,2144415941578981993,computers.desktop,multitronics,126.87,1515915625572481386,hwFP0wJ6ge
6405,2021-01-19 17:12:14 UTC,view,453830,2144415941578981993,computers.desktop,multitronics,94.00,1515915625594956296,W39zhcnx2G
6741,2021-01-25 05:31:20 UTC,view,453833,2144415941578981993,computers.desktop,multitronics,62.84,1515915625596805254,edMLg7NrgD
7441,2021-02-05 08:06:15 UTC,view,453814,2144415941578981993,computers.desktop,multitronics,65.71,1515915625600833693,lTRsygfYJl
7696,2021-02-09 17:16:12 UTC,view,453808,2144415941578981993,computers.desktop,multitronics,93.13,1515915625603383316,4Y1fiCIh5H
8545,2021-02-23 09:56:23 UTC,view,453809,2144415941578981993,computers.desktop,multitronics,72.83,1515915625608690404,Ld0uoVC2Gj


## 연습 공간

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dbpath = "/content/drive/MyDrive/sql/user_log.db"
conn = sqlite3.connect(dbpath)
def SQL(query):
  cur = conn.cursor()
  cur.execute(query)
  df = pd.read_sql_query(query, conn)
  # return cur.fetchall()
  return df

In [ ]:
query =\
'''

SELECT A.event_time
FROM user_data AS A
WHERE 1=1
-- AND event_type = 'view'
AND category_code is null
ORDER BY event_time


;'''
SQL(query)

,event_time
0,2020-09-24 13:46:10+00:00
1,2020-09-24 13:56:06+00:00
2,2020-09-24 14:44:47+00:00
3,2020-09-24 17:24:34+00:00
4,2020-09-24 18:13:30+00:00
...,...
2417,2021-02-28 19:22:30+00:00
2418,2021-02-28 20:56:47+00:00
2419,2021-02-28 21:02:58+00:00
2420,2021-02-28 22:06:32+00:00
